In [ ]:
import os
import numpy as np
from PIL import Image
import nibabel as nib

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
dataset_path = '/content/drive/MyDrive/ADNI_nii_after_pre'

In [ ]:
class ImageSlicer:
    def __init__(self, dataset_path):
        # Set the path to the dataset
        self.dataset_path = dataset_path
        # Define the labels for the dataset
        self.labels = ['AD', 'CN', 'LMCI', 'MCI']
        # Set the path to the output folder for the sliced images
        self.output_folder = '/content/drive/MyDrive/output_images_preprocessed'
        # Set the range of slices to extract from each image
        self.slices_range = range(10)

    def slice_images(self):
        # Iterate over each label in the dataset
        for label in self.labels:
            # Set the input folder path for the label
            input_folder = os.path.join(self.dataset_path, label)
            # Set the output folder path for the label
            output_folder = os.path.join(self.dataset_path, self.output_folder, label)
            # Create the output folder if it doesn't exist
            os.makedirs(output_folder, exist_ok=True)

            # Iterate over each file in the input folder
            for filename in os.listdir(input_folder):
                # Check if the file has the ".nii" extension
                if filename.endswith('.nii'):
                    # Set the input file path
                    filepath = os.path.join(input_folder, filename)
                    # Load the NIfTI image using the nibabel library
                    nii_image = nib.load(filepath)
                    # Convert the image to a NumPy array and add an additional dimension along axis=-1
                    np_image = np.array(nii_image.get_fdata())[..., np.newaxis]
                    # Remove the additional dimension along axis=-1
                    np_image = np.squeeze(np_image, axis=-1)
                    # Cast the NumPy array to the float32 data type
                    np_image = np_image.astype(np.float32)
                    # Normalize the image to the range [0, 255]
                    min_val = np.min(np_image)
                    max_val = np.max(np_image)
                    np_image = ((np_image - min_val) * (255 / (max_val - min_val))).astype(np.uint8)
                    # Get the number of slices in the image 3D
                    num_slices = np_image.shape[2]
                    # Get the middle slice index
                    middle_slice = num_slices // 2
                    # Get the starting and ending slice indices
                    start_slice = max(0, middle_slice - 5)
                    end_slice = min(num_slices - 1, middle_slice + 4)
                    # Create a folder for the image slices
                    folder_name = os.path.splitext(filename)[0] + '_folder_0'
                    folder_path = os.path.join(output_folder, folder_name)
                    os.makedirs(folder_path, exist_ok=True)
                    # Extract and save each slice within the specified range
                    for i in range(start_slice, end_slice+1):
                        # Extract the slice as a PIL Image object
                        slice_image = Image.fromarray(np_image[:, :, i])
                        # Convert the slice to the RGB color mode
                        slice_image = slice_image.convert('RGB')
                        # Set the filename for the slice image
                        slice_filename = f'slice_{i}.jpg'
                        # Set the filepath for the slice image
                        slice_filepath = os.path.join(folder_path, slice_filename)
                        # Save the slice image as a JPEG file
                        slice_image.save(slice_filepath)


In [ ]:
# create an instance of the ImageSlicer class with the path to the dataset
slicer = ImageSlicer(dataset_path)

# call the slice_images method to slice and store the images
slicer.slice_images()
